In [ ]:
# !pip install pymysql
# !pip install asyncio
# !pip install websockets
# !pip install joblib

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pymysql
from pymysql.cursors import DictCursor
import pandas as pd
import numpy as np
import joblib
from decimal import Decimal
from sklearn.preprocessing import MinMaxScaler
import time
import json
import asyncio
import websockets

### DB연결, 훈련모델 불러오기

In [ ]:
# DB 연결
conn = pymysql.connect(host='project-db-stu.ddns.net', user='insogae', password='insogae', db='insogae', port=3307, charset='utf8')

# 사용자 ID를 입력 받고 사용자의 수조 리스트를 배열에 담기
cursor = conn.cursor()
sql = "select TANK_ID from TANK where COMPANY_ID = 'insogae'" # 로그인시 들어갈 ID
cursor.execute(sql)
rows = cursor.fetchall()
cursor.close()
print(rows)
tankList = []
for row in rows:
    tankList.append(row[0])
model_test = joblib.load('./model/wt11_do_model.pkl')  
# 훈련시킨 모델마다 다 다른결과가 나옴 하지만 큰차이가 없는 결과로 하나만쓰기로 함.
# 차후 6시간짜리를 10분짜리 훈련시킨걸로 예측값 테스트했을때 좋지 않을경우 6시간짜리로 대체할 예정!
tankList

### 수조데이터 delete함수

In [ ]:
# 각 수조의 첫 12행을 제외한 나머지 DELETE 시키기!!

# 데이터베이스 연결
def deleteData():
    cursor = conn.cursor()
    # 각 탱크마다 쿼리를 생성하여 실행
    for tank_name in tankList:
        # 첫 15개의 레코드를 제외한 나머지 레코드 삭제 (LIMIT 15)
        query = '''
        DELETE t1 FROM TANK_DATA_copy t1
        WHERE RECORD_SEQ NOT IN (
            SELECT t2.RECORD_SEQ FROM (
                SELECT RECORD_SEQ FROM TANK_DATA_copy
                WHERE TANK_ID = %s ORDER BY RECORD_DATE ASC LIMIT 2812
            ) AS t2
        )
        AND TANK_ID = %s
        '''
        
        cursor.execute(query, (tank_name, tank_name))

    # 변경 내용을 커밋하고 연결 종료
    cursor.close()
    conn.commit()
    cursor2 = conn.cursor()
    sql = "select count(*) from TANK_DATA_copy"
    cursor2.execute(sql)
    rowCnt = cursor2.fetchall()
    print(rowCnt)
    cursor2.close()

In [ ]:
# 12개 행 남기고 다 지우기

deleteData()

### 10분단위 예측값 핸들링(DB insert, Json데이터로 반환)

In [ ]:
# 예측한 데이터 집어넣기!!  (10분짜리)

conn = pymysql.connect(host='project-db-stu.ddns.net', user='insogae', password='insogae', db='insogae', port=3307, charset='utf8')
count = 2800; # 4500번째 부터 카운트해서 예측할 예정 즉 TANK_DATA_copy 가 약 36,000 개 정도 예측이 되어야함.

import threading
import time

lock = threading.Lock()
timer = None

def insertData():
    global count
    data_list_of_tankList = []
    testList = []
    for tank_name in tankList:
        cursor = conn.cursor(DictCursor)
        sql = "select DO,PH,TEMP,SALT,TANK_ID,RECORD_DATE from TANK_DATA where TANK_ID ='{tank_id}' order by RECORD_DATE limit {count},13"
        sql = sql.format(tank_id=tank_name, count=count)
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            for key, value in row.items():
                if isinstance(value, Decimal):
                    row[key] = float(str(value))
        testList.append(rows)
        data_list_of_tankList.append([[list(row.values()) for row in rows]])
        cursor.close()
    accListSet = []
    for i in testList:
        accListSet.append(i[12])
    accListSet
    new_list = [[[elem[:-2] for elem in subsublist] for subsublist in sublist] for sublist in data_list_of_tankList]
    for i in range(len(new_list)):
        for j in range(len(new_list[0])):
            new_list[i][j] = new_list[i][j][:12]
    count = count+1
    x = []
    scaler = MinMaxScaler()
    resultDataList = [] # 예측후에 실제값과 예측값 및 측정 시간을 담을 배열 
    for i, tank in enumerate(new_list):
        x.append(tank[0])
        scaler_data = scaler.fit_transform(x[i]) # 독립변수 스케일링
        reshaped_data = scaler_data.reshape(-1,12,4) # shape 변경
        pre = model_test.predict(reshaped_data) # 모델 예측
        reshaped_array = np.tile(pre, (1, 4)) # shape 재설정
        pre_data = scaler.inverse_transform(reshaped_array)  # 실제 예측값 shape = (1,4) // DO, PH, TEMP, SALT
        cursor = conn.cursor()
        sql = f"""insert into TANK_DATA_copy 
                    VALUES (NULL,'{tankList[i]}',{pre_data[0][0]},{pre_data[0][1]},{pre_data[0][2]},{pre_data[0][3]},
                    (SELECT max(record_date)
                    FROM    (SELECT a.*,
                                ROW_NUMBER() OVER (PARTITION BY a.TANK_ID ORDER BY a.RECORD_DATE desc )  AS row_num 
                            from TANK_DATA_copy a ,TANK b
                            where a.TANK_ID = b.TANK_ID
                            and b.COMPANY_ID = 'insogae') sub 
                    where sub.row_num <= 1
                    and sub.TANK_ID = '{tankList[i]}'
                    order by sub.TANK_ID,sub.RECORD_DATE)
                    + INTERVAL 10 MINUTE)"""
        cursor.execute(sql)
        cursor.close()
        conn.commit()
        resultDataList.append(pre_data[0][0])
        resultDataList.append(pre_data[0][1])
        resultDataList.append(pre_data[0][2])
        resultDataList.append(pre_data[0][3])

    result=[]
    tank_dict = {}
    for i, val in enumerate(resultDataList):
        if i % 4 == 0:
            if tank_dict:
                result.append(tank_dict)
                tank_dict = {}
            tank_dict['DO_PRE'] = val
        elif i % 4 == 1:
            tank_dict['PH_PRE'] = val
        elif i % 4 == 2:
            tank_dict['TEMP_PRE'] = val
        elif i % 4 == 3:
            tank_dict['SALT_PRE'] = val

    result.append(tank_dict)
    for i in range(len(result)):
        accListSet[i].update(result[i])
    print(result)


    for d in accListSet:
        d['RECORD_DATE'] = d['RECORD_DATE'].isoformat()
        d['DO'] = str(d['DO'])
        d['PH'] = str(d['PH'])
        d['TEMP'] = str(d['TEMP'])
        d['SALT'] = str(d['SALT'])
        d['DO_PRE'] = str(d['DO_PRE'])
        d['PH_PRE'] = str(d['PH_PRE'])
        d['SALT_PRE'] = str(d['SALT_PRE'])
        d['TEMP_PRE'] = str(d['TEMP_PRE'])
    
    tempData = {"type":"insertData","value":accListSet}
    jsonData = json.dumps(tempData)
    return jsonData

### 6시간단위 예측 함수

#### 6시간단위 예측함수 정의

In [ ]:
# 6시간 후 예측하기위한 함수 정의

def preData(dateTime, tankName):
    data_list_of_tankList = []
    testList = []
    for tank_name in tankList:
        cursor = conn.cursor(DictCursor)
        sql = """SELECT *
                 FROM (
                 SELECT *, ROW_NUMBER() OVER (PARTITION BY TANK_ID ORDER BY RECORD_DATE) AS row_num
                 FROM TANK_DATA
                 WHERE RECORD_DATE 
                        IN ('{dateTime}', 
                            DATE_SUB('{dateTime}', INTERVAL 6 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 12 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 18 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 24 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 30 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 36 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 42 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 48 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 54 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 60 HOUR),
                            DATE_SUB('{dateTime}', INTERVAL 66 HOUR))
                 ) AS t"""
        sql = sql.format(dateTime=dateTime)
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            for key, value in row.items():
                if isinstance(value, Decimal):
                    row[key] = float(str(value))
        testList.append(rows)
        data_list_of_tankList.append([[list(row.values()) for row in rows]])
        cursor.close()
    accListSet = []
    for i in testList:
        accListSet.append(i[12])
    accListSet
    new_list = [[[elem[2:-2] for elem in subsublist] for subsublist in sublist] for sublist in data_list_of_tankList]
    for i in range(len(new_list)):
        for j in range(len(new_list[0])):
            new_list[i][j] = new_list[i][j][:12]
    scaler = MinMaxScaler()
    new_pre_data = [] # 예측후에 실제값과 예측값 및 측정 시간을 담을 배열 
    x = []
    for i, tank in enumerate(new_list):
        x.append(tank[0])
        scaler_data = scaler.fit_transform(x[i]) # 독립변수 스케일링
        reshaped_data = scaler_data.reshape(-1,12,4) # shape 변경
        pre = model_test.predict(reshaped_data) # 모델 예측              
        reshaped_array = np.tile(pre, (1, 4)) # shape 재설정
        pre_data = scaler.inverse_transform(reshaped_array)  # 실제 예측값 shape = (1,4) // DO, PH, TEMP, SALT
        print([tankList[i]])
        new_pre_data.append(np.concatenate((np.array([[tankList[i]]]), pre_data), axis=1))
    new_pre_data
    preDataDictList = []
    for row in new_pre_data:
        my_dict = {
            "TANK_ID": row[0][0],
            "DO": float(row[0][1]),
            "PH": float(row[0][2]),
            "TEMP": float(row[0][3]),
            "SALT": float(row[0][4])
        }
        preDataDictList.append(my_dict)
    
    class CustomJSONEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, datetime):
                return obj.isoformat()
            elif isinstance(obj, (int, float)):
                return str(obj)
            else:
                return super().default(obj)
    json_data = json.dumps(preDataDictList, cls=CustomJSONEncoder)
    tempData = {"type":"predictData","value":json_data}
    jsonData = json.dumps(tempData)
    print(jsonData)
    return jsonData


#### 6시간단위 예측함수로부터 Json데이터 반환

In [ ]:
# 10분단위 예측값 insert 할때, 같은 시간대에서 6시간후를 예측하기 위해 같은 시간대와, 수조이름을 뽑아내기 위한 함수 정의
def predictData():
    global count
    data_list_of_tankList = []
    testList = []
    for tank_name in tankList:
        cursor = conn.cursor(DictCursor)
        sql = "select DO,PH,TEMP,SALT,TANK_ID,RECORD_DATE from TANK_DATA where TANK_ID ='{tank_id}' order by RECORD_DATE limit {count},13"
        sql = sql.format(tank_id=tank_name, count=count)
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            for key, value in row.items():
                if isinstance(value, Decimal):
                    row[key] = float(str(value))
        testList.append(rows)
        data_list_of_tankList.append([[list(row.values()) for row in rows]])
        cursor.close()
    accListSet = []
    for i in testList:
        accListSet.append(i[12])
    print(accListSet[0]['RECORD_DATE'])
    return preData(accListSet[0]['RECORD_DATE'], accListSet[0]['TANK_ID']) ## 독립변수를 집어넣고 데이터 예측하기!! 
     

    

### 이상치 점검후 Json데이터 반환

In [ ]:
# 이상치 점검 함수
def insertDangerData():
    data_list_of_tankList = []
    testList = []
    for tank_name in tankList:
        cursor = conn.cursor(DictCursor)
        sql = "select DO,PH,TEMP,SALT,TANK_ID,RECORD_DATE from TANK_DATA where TANK_ID ='{tank_id}' order by RECORD_DATE limit {count},1"
        sql = sql.format(tank_id=tank_name, count=count+11)
        cursor.execute(sql)
        rows = cursor.fetchall()
        for row in rows:
            for key, value in row.items():
                if isinstance(value, Decimal):
                    row[key] = float(str(value))
        testList.append(rows)
        data_list_of_tankList.append([[list(row.values()) for row in rows]])
        cursor.close()
    # 이상치 를 담을 리스트
    dangerValueList = []


    for i in testList:
    #     print(i[0])
        # 이상치 비교 하기
        if(i[0]['TEMP']<23 or i[0]['TEMP']>32):
            dangerDict ={
                'TANK_ID' : i[0]['TANK_ID'],
                'VALUE': i[0]['TEMP'],
                'TYPE': list(i[0].keys())[2],
                'RECORD_DATE': i[0]['RECORD_DATE']
            }
            dangerValueList.append(dangerDict)
        if(i[0]['DO']<4 or i[0]['DO']>10):
            dangerDict ={
                'TANK_ID' : i[0]['TANK_ID'],
                'VALUE': i[0]['DO'],
                'TYPE': list(i[0].keys())[0],
                'RECORD_DATE': i[0]['RECORD_DATE']
            }
            dangerValueList.append(dangerDict)
        if(i[0]['PH']<5.5 or i[0]['PH']>8.5):
            dangerDict ={
                'TANK_ID' : i[0]['TANK_ID'],
                'VALUE': i[0]['PH'],
                'TYPE': list(i[0].keys())[1],
                'RECORD_DATE': i[0]['RECORD_DATE']
            }
            dangerValueList.append(dangerDict)
        if(i[0]['SALT']<0.05 or i[0]['SALT']>5.34):
            dangerDict ={
                'TANK_ID' : i[0]['TANK_ID'],
                'VALUE': i[0]['SALT'],
                'TYPE': list(i[0].keys())[3],
                'RECORD_DATE': i[0]['RECORD_DATE']
            }
            dangerValueList.append(dangerDict)
    dangerValueList[0]['VALUE']
    # 이상치 점검해서 리스트에 담음!!
    # 만약 이상치 (dangerValueList)가 null이 아닐경우 이상치를 DB(테이블 : Alarm) 에 넣기


    for dict_ in dangerValueList:
        dict_['RECORD_DATE'] = dict_['RECORD_DATE'].strftime('%Y-%m-%d %H:%M:%S')

    # convert list to JSON
    json_str = json.dumps(dangerValueList)

    if dangerValueList is not None:
        for i in dangerValueList:
            cursor = conn.cursor()
            sql = f"""insert into ALARM 
                            VALUES (NULL,
                                    '{i['TANK_ID']}',
                                    '{i['VALUE']}',
                                    '{i['RECORD_DATE']}',
                                    'insogae',
                                    '{i['TYPE']}')

                  """
            cursor.execute(sql)
            cursor.close()
        conn.commit()
        # convert datetime objects to string format
        json_str
        tempData = {"type":"dangerData","value":json_str}
        jsonData = json.dumps(tempData)
        return jsonData
    # dangerValueList가 null일경우 None 반환
    else:
        return None

### DB 연결실패시 바로 연결

In [ ]:
conn = pymysql.connect(host='project-db-stu.ddns.net', user='insogae', password='insogae', db='insogae', port=3307, charset='utf8')

### 파이썬웹소켓서버 구동 및 함수 실행-> Json데이터 클라이언트 전송

In [ ]:
import asyncio
import websockets
import nest_asyncio
nest_asyncio.apply()


async def server(websocket, path):
    while True:
        data = await websocket.recv()
        print(data)
        response = "Hello, client!"
        task = asyncio.create_task(sendData(websocket))
        await task
        
async def sendData(websocket):
    while True:
#         test = {"type":"test","value":'테스트'} # 여러 json을 보내기 위한 테스트
#         jsonTest = json.dumps(test) 
        preData6hour = predictData() # 예측값을 통해서 6시간뒤 보여주기위한 데이터 내보내기
        dangerData = insertDangerData() # 이상치를 DB에 insert 함과 동시에 클라이언트로 데이터 보내기 (토스트에 활용)
        updateData = insertData() # 10분단위 예측값을 DB에 insert 함과 동시에 클라이언트로 데이터 보내기 (대시보드, 시각화)
        # 만약 이상치데이터가 None이 아닐경우에만 데이터 보내기
        if dangerData is not None:
            await websocket.send(dangerData)
        await websocket.send(preData6hour)
        await websocket.send(updateData)
#         await websocket.send(jsonTest)
        await asyncio.sleep(3)
        
async def main():
    async with websockets.serve(server, 'localhost', 8123):
        await asyncio.Future()  # 서버가 종료되지 않도록 무한대기

asyncio.run(main())